# NVIDIA 

The `langchain-nvidia-ai-endpoints` package contains LangChain integrations for chat models and embeddings powered by [NVIDIA AI Foundation Models](https://www.nvidia.com/en-us/ai-data-science/foundation-models/), and hosted on the [NVIDIA API Catalog](https://build.nvidia.com/).

NVIDIA AI Foundation models are community- and NVIDIA-built models that are optimized to deliver the best performance on NVIDIA-accelerated infrastructure. 
You can use the API to query live endpoints that are available on the NVIDIA API Catalog to get quick results from a DGX-hosted cloud compute environment, 
or you can download models from NVIDIA's API catalog with NVIDIA NIM, which is included with the NVIDIA AI Enterprise license. 
The ability to run models on-premises gives your enterprise ownership of your customizations and full control of your IP and AI application. 

NIM microservices are packaged as container images on a per model/model family basis 
and are distributed as NGC container images through the [NVIDIA NGC Catalog](https://catalog.ngc.nvidia.com/). 
At their core, NIM microservices are containers that provide interactive APIs for running inference on an AI Model. 

Use this documentation to learn how to install the `langchain-nvidia-ai-endpoints` package 
and use it for some common functionality for text-generative and embedding models.

## Install the Package

In [ ]:
%pip install --upgrade --quiet  langchain-nvidia-ai-endpoints

## Access the NVIDIA API Catalog

To get access to the NVIDIA API Catalog, do the following:

1. Create a free account on the [NVIDIA API Catalog](https://build.nvidia.com/) and log in.
2. Click your profile icon, and then click **API Keys**. The **API Keys** page appears.
3. Click **Generate API Key**. The **Generate API Key** window appears.
4. Click **Generate Key**. You should see **API Key Granted**, and your key appears.
5. Copy and save the key as `NVIDIA_API_KEY`.
6. To verify your key, use the following code.

In [4]:
import getpass
import os

if os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    print("Valid NVIDIA_API_KEY already in environment. Delete to reset")
else:
    nvapi_key = getpass.getpass("NVAPI Key (starts with nvapi-): ")
    assert nvapi_key.startswith(
        "nvapi-"
    ), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key

You can now use your key to access endpoints on the NVIDIA API Catalog.

## Work with the API Catalog

To submit a query to the [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5/modelcard) model,
run the following code. If you don't specify a model, the embedder uses the default model.

In [ ]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

# embedder.get_available_models()

embedder = NVIDIAEmbeddings(model="nvidia/nv-embedqa-e5-v5")

### Similarity

The following example prepares and demonstrates a similarity check by using the following data:

- Queries

  - What's the weather like in Komchatka?

  - What kinds of food is Italy known for?

  - What's my name? I bet you don't remember...

- Documents

  - Komchatka's weather is cold, with long, severe winters.

  - Italy is famous for pasta, pizza, gelato, and espresso.

  - I can't recall personal names, only provide information.

#### Embedding Runtimes

In [ ]:
print("\nSequential Embedding: ")
q_embeddings = [
    embedder.embed_query("What's the weather like in Komchatka?"),
    embedder.embed_query("What kinds of food is Italy known for?"),
    embedder.embed_query("What's my name? I bet you don't remember...")
]
print("Shape:", (len(q_embeddings), len(q_embeddings[0])))

#### Document Embedding

In [ ]:
print("\nBatch Document Embedding: ")
d_embeddings = embedder.embed_documents(
    [
        "Komchatka's weather is cold, with long, severe winters.",
        "Italy is famous for pasta, pizza, gelato, and espresso.",
        "I can't recall personal names, only provide information."
    ]
)
print("Shape:", (len(q_embeddings), len(q_embeddings[0])))

Now that we've generated our embeddings, we can do a similarity check on the results to see which documents would have triggered as reasonable answers in a retrieval task.

In [ ]:
%pip install --upgrade --quiet  matplotlib scikit-learn

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Compute the similarity matrix between q_embeddings and d_embeddings
cross_similarity_matrix = cosine_similarity(
    np.array(q_embeddings),
    np.array(d_embeddings),
)

# Plotting the cross-similarity matrix
plt.figure(figsize=(8, 6))
plt.imshow(cross_similarity_matrix, cmap="Greens", interpolation="nearest")
plt.colorbar()
plt.title("Cross-Similarity Matrix")
plt.xlabel("Query Embeddings")
plt.ylabel("Document Embeddings")
plt.grid(True)
plt.show()

### Truncation

Embedding models typically have a fixed context window that determines the maximum number of input tokens that can be embedded. This limit can be a hard limit, equal to the model's maximum input token length, or an effective limit, beyond which the accuracy of the embedding decreases. Since models operate on tokens, and applications usually work with text, it can be challenging for an application to ensure that its input stays within the model's token limits. By default, an exception is thrown if the input is too large.

NVIDIA NIM microservices can truncate the input on the server side if it's too large. The `truncate` parameter accepts the following values:

 - **NONE** – An exception is thrown if the input is too large. This is the default option. 
 - **START** – The server truncates from the start of the input, discarding tokens as necessary.
 - **END** – The server truncates from the end of the input, discarding tokens as necessary.

Use the following example to create some long text and then embed it with and without truncating the input.

In [15]:
long_text = "AI is amazing, amazing is " * 100

In [ ]:
strict_embedder = NVIDIAEmbeddings()
try:
    strict_embedder.embed_query(long_text)
except Exception as e:
    print("Error:", e)

In [ ]:
truncating_embedder = NVIDIAEmbeddings(truncate="END")
truncating_embedder.embed_query(long_text)[:5]

## Self-host with NVIDIA NIM Microservices

When you are ready to deploy your AI application, you can self-host models with NVIDIA NIM. 
For more information, refer to [NVIDIA NIM Microservices](https://www.nvidia.com/en-us/ai-data-science/products/nim-microservices/).

The following code connects to locally hosted NIM Microservices.

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings, NVIDIARerank

# connect to an chat NIM running at localhost:8000, specifying a specific model
llm = ChatNVIDIA(base_url="http://localhost:8000/v1", model="meta/llama3-8b-instruct")

# connect to an embedding NIM running at localhost:8080
embedder = NVIDIAEmbeddings(base_url="http://localhost:8080/v1")

# connect to a reranking NIM running at localhost:2016
ranker = NVIDIARerank(base_url="http://localhost:2016/v1")

## Related Topics

- [langchain-nvidia-ai-endpoints package ReadMe](https://github.com/langchain-ai/langchain-nvidia/blob/main/libs/ai-endpoints/README.md)
- [Overview of NVIDIA NIM for Large Language Models (LLMs)](https://docs.nvidia.com/nim/large-language-models/latest/introduction.html)
- [Overview of NeMo Retriever Embedding NIM](https://docs.nvidia.com/nim/nemo-retriever/text-embedding/latest/overview.html)
- [Overview of NeMo Retriever Reranking NIM](https://docs.nvidia.com/nim/nemo-retriever/text-reranking/latest/overview.html)